Install the package via "pip install opencosmorspy"

Import the module

In [1]:
import numpy as np

from opencosmorspy import COSMORS

Choose a parameterization (default_turbomole or default_orca). You can either instanciate a parameterization object and pass it to the COSMORS constructor, or let the COSMORS class take over the instanciation. To use the newest parameterization please check last exmaple.

In [2]:
crs = COSMORS(par='default_orca')

To modify parameters, currently indivual properties of the parameters class are overwritten. 

In [3]:
crs.par.calculate_contact_statistics_molecule_properties = True

In [4]:
print(crs.par)

default_orca:
                                      qc_program : orca
                                  descriptor_lst : ['sigma', 'sigma_orth', 'elmnt_nr', 'group', 'mol_charge']
                                           a_eff : 6.226
                                            r_av : 0.5
                                       sigma_min : -0.15
                                       sigma_max : 0.15
                                      sigma_step : 0.001
                                      sigma_grid : [-1.50000000e-01 -1.49000000e-01 -1.48000000e-01 -1.47000000e-01
 -1.46000000e-01 -1.45000000e-01 -1.44000000e-01 -1.43000000e-01
 -1.42000000e-01 -1.41000000e-01 -1.40000000e-01 -1.39000000e-01
 -1.38000000e-01 -1.37000000e-01 -1.36000000e-01 -1.35000000e-01
 -1.34000000e-01 -1.33000000e-01 -1.32000000e-01 -1.31000000e-01
 -1.30000000e-01 -1.29000000e-01 -1.28000000e-01 -1.27000000e-01
 -1.26000000e-01 -1.25000000e-01 -1.24000000e-01 -1.23000000e-01
 -1.22000000e-01 -1.21000000e-01

Add a molecule:

In [5]:
mol_structure_list_0 = ['./../tests/COSMO_ORCA/C2H2Cl4_001_1112tetrachloroethane/COSMO_TZVPD/C2H2Cl4_001_1112tetrachloroethane_CnfS1_c000.orcacosmo']
crs.add_molecule(mol_structure_list_0)

You can clear the list of molecules at any stage:

In [6]:
crs.clear_molecules()

But now we do want to add two molecules. At present, only one molecular structure per molecule is supported.

In [7]:
mol_structure_list_0 = ['./../tests/COSMO_ORCA/C2H2Cl4_001_1112tetrachloroethane/COSMO_TZVPD/C2H2Cl4_001_1112tetrachloroethane_CnfS1_c000.orcacosmo']
crs.add_molecule(mol_structure_list_0)
crs.add_molecule(['./../tests/COSMO_ORCA/H2O/COSMO_TZVPD/H2O_c000.orcacosmo'])

Let us add two jobs:

In [8]:
x = np.array([0.0, 1.0])
T = 298.15
crs.add_job(x, T, refst='pure_component')

In [9]:
x = np.array([0.5, .5])
T = 298.15
crs.add_job(x, T, refst='pure_component')

In [10]:
results = crs.calculate()

In [11]:
print('Total logarithmic activity coefficient: ', results['tot']['lng'])
print('Residual logarithmic activity coefficient: ', results['enth']['lng'])
print('Combinatorial logarithmic activity coefficient:', results['comb']['lng'])

Total logarithmic activity coefficient:  [[8.68627398 0.        ]
 [1.04077495 1.53128118]]
Residual logarithmic activity coefficient:  [[9.73709277 0.        ]
 [1.15474901 1.76906839]]
Combinatorial logarithmic activity coefficient: [[-1.05081879  0.        ]
 [-0.11397406 -0.23778722]]


To view partial molar properties and average interaction energies:

In [12]:
print('Partial molar           - pm_A_int: \n', results['enth']['pm_A_int'], '\n')
print('Partial molar           - pm_E_int: \n', results['enth']['pm_E_int'], '\n')
print('Partial molar           - pm_E_hb: \n', results['enth']['pm_E_hb'], '\n')
print('Partial molar           - pm_E_mf: \n', results['enth']['pm_E_mf'], '\n')
print('Average interaction mol - aim_A_int: \n', results['enth']['aim_A_int'], '\n')

Partial molar           - pm_A_int: 
 [[5667.60682391    0.        ]
 [1182.4581352  1080.3155726 ]] 

Partial molar           - pm_E_int: 
 [[9311.12171104    0.        ]
 [2142.0133706  1679.75869705]] 

Partial molar           - pm_E_hb: 
 [[6072.5248119     0.        ]
 [1599.25872566  999.07187408]] 

Partial molar           - pm_E_mf: 
 [[3238.59689915    0.        ]
 [ 542.75464493  680.68682297]] 

Average interaction mol - aim_A_int: 
 [[3694.6397339     0.        ]
 [ 131.66530029 2131.10833209]] 



You may also use different reference states for the molecules. To use a fixed concentration referene mixture:

In [ ]:
crs.clear_jobs()

x = np.array([0.0, 1.0])
T = 298.15
crs.add_job(x, T, refst='reference_mixture', x_refst=np.array([0, 1.]))

x = np.array([0.5, 0.5])
T = 298.15
crs.add_job(x, T, refst='reference_mixture', x_refst=np.array([0, 1.]))

results = crs.calculate()

print('Total logarithmic activity coefficient: \n', results['tot']['lng'], '\n')

Total activity coefficient: 
 [[ 0.          0.        ]
 [-7.64549903  1.53128118]] 



To use the COSMO reference state directly:

In [14]:
crs.clear_jobs()
x = np.array([0.5, 0.5])
T = 298.15
crs.add_job(x, T, refst='cosmo', x_refst=np.array([0, 1.]))
results = crs.calculate()

crs.clear_jobs()
x = np.array([0.0, 1.0])
T = 298.15
crs.add_job(x, T, refst='cosmo', x_refst=np.array([0, 1.]))
results_ref = crs.calculate()

In [ ]:
print('Total logarithmic activity coefficient: \n', results['tot']['lng']-results_ref['tot']['lng'], '\n')

Total activity coefficient: 
 [[-7.64549903  1.53128118]] 



Now to use the newest [openCOSMO-RS 24a](https://doi.org/10.1016/j.fluid.2024.114250) parameter set:

(keep in mind that results may vary slightly because the implementations are a little different)

In [17]:
import numpy as np
from opencosmorspy.parameterization import openCOSMORS24a
from opencosmorspy.cosmors import COSMORS
crs = COSMORS(par=openCOSMORS24a())

crs.add_molecule(['./../tests/COSMO_ORCA/cyclohexane.orcacosmo'])
crs.add_molecule(['./../tests/COSMO_ORCA/acetic_acid.orcacosmo'])

# infinite dilution activity coefficient of cyclohexane in acetic acid at 25 C
x = np.array([0.0, 1.0])
T = 298
crs.add_job(x, T, refst='pure_component')

# infinite dilution activity coefficient of cyclohexane in acetic acid at 45 C
x = np.array([0.0, 1.0])
T = 318
crs.add_job(x, T, refst='pure_component')

# 50:50 mole fraction based mixture of cyclohexane and acetic acid
x = np.array([0.5, .5])
T = 298.15
crs.add_job(x, T, refst='pure_component')

results = crs.calculate()

print('Total logarithmic activity coefficient: ', results['tot']['lng'], '\n')

print('Residual logarithmic activity coefficient: ', results['enth']['lng'])
print('Combinatorial logarithmic activity coefficient:', results['comb']['lng'])

Total logarithmic activity coefficient:  [[3.18179476 0.        ]
 [3.02591755 0.        ]
 [0.88209786 0.73057466]] 

Residual logarithmic activity coefficient:  [[3.16087856 0.        ]
 [3.00500135 0.        ]
 [0.88285676 0.72645049]]
Combinatorial logarithmic activity coefficient: [[ 0.0209162   0.        ]
 [ 0.0209162   0.        ]
 [-0.0007589   0.00412417]]
